In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fastapi uvicorn pyngrok streamlit nest_asyncio sqlalchemy passlib    --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.7 MB/s eta 0:00:00


In [3]:
from pyngrok import ngrok
ngrok.set_auth_token("34xW6jzwinF6oZ2xR6SdnMD7fxp_6y54Mk6zsPEt7JN8fx46")

In [4]:
import sqlite3
from passlib.context import CryptContext

DB_PATH = "/content/drive/MyDrive/students.db"
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

c.execute('''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL,
    role TEXT NOT NULL CHECK(role IN ('student', 'admin'))
)
''')

# Ensure correct structure
c.execute('''
CREATE TABLE IF NOT EXISTS study_log (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    date DATETIME DEFAULT CURRENT_TIMESTAMP,
    study_hours REAL NOT NULL,
    social_media_hours REAL NOT NULL,
    sleep_hours REAL NOT NULL,
    stress_level INTEGER NOT NULL,
    exam_score REAL,
    notes TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
)
''')

conn.commit()
conn.close()
print("Database fixed! You can now add logs.")

# Re-open connection for admin creation
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

# Create default admin if not exists
c.execute("SELECT * FROM users WHERE email = 'admin@college.com'")
if not c.fetchone():
    hashed = pwd_context.hash("admin123")
    c.execute("INSERT INTO users (name, email, password, role) VALUES (?, ?, ?, ?)",
              ("Admin", "admin@college.com", hashed, "admin"))
    print("Default Admin Created → admin@college.com / admin123")

conn.commit()
conn.close()
print("Database ready!")

Database fixed! You can now add logs.
Database ready!


In [5]:
%%writefile backend.py
from fastapi import FastAPI, HTTPException, Header, Depends
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional
import sqlite3
from passlib.context import CryptContext
import re

app = FastAPI()
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

DB_PATH = "/content/drive/MyDrive/students.db"

def valid_email(email: str) -> bool:
    return re.match(r"^[\w\.\-]+@[\w\.\-]+\.\w+$", email) is not None

# Models
class UserIn(BaseModel):
    name: str
    email: str
    password: str
    role: str

class LoginIn(BaseModel):
    email: str
    password: str
    role: str

class StudyLogIn(BaseModel):
    study_hours: float
    social_media_hours: float
    sleep_hours: float
    stress_level: int
    exam_score: Optional[float] = None
    notes: Optional[str] = None

def get_current_user(x_user_id: Optional[str] = Header(None)):
    if not x_user_id:
        raise HTTPException(401, "Missing user ID")
    try:
        return int(x_user_id)
    except:
        raise HTTPException(401, "Invalid user ID")

@app.post("/signup")
def signup(u: UserIn):
    if u.role not in ["student", "admin"]:
        raise HTTPException(400, "Invalid role")
    if not valid_email(u.email):
        raise HTTPException(400, "Invalid email")

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id FROM users WHERE email = ?", (u.email,))
    if cur.fetchone():
        conn.close()
        raise HTTPException(400, "Email already registered")

    hashed = pwd_context.hash(u.password)
    cur.execute("INSERT INTO users (name, email, password, role) VALUES (?, ?, ?, ?)",
                (u.name, u.email, hashed, u.role))
    conn.commit()
    conn.close()
    return {"message": "Signup successful"}

@app.post("/login")
def login(u: LoginIn):
    if not valid_email(u.email):
        raise HTTPException(400, "Invalid email")

    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id, name, email, password, role FROM users WHERE email = ? AND role = ?",
                (u.email, u.role))
    user = cur.fetchone()
    conn.close()

    if not user or not pwd_context.verify(u.password, user[3]):
        raise HTTPException(401, "Wrong credentials")

    return {"message": "Login OK", "user": {"id": user[0], "name": user[1], "email": user[2], "role": user[4]}}

@app.get("/students")
def get_students():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id, name, email FROM users WHERE role = 'student'")
    rows = cur.fetchall()
    conn.close()
    return [{"id": r[0], "name": r[1], "email": r[2]} for r in rows]

@app.post("/add-user")
def add_user(u: UserIn):
    if not valid_email(u.email):
        raise HTTPException(400, "Invalid email")
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id FROM users WHERE email = ?", (u.email,))
    if cur.fetchone():
        conn.close()
        raise HTTPException(400, "Email already exists")
    hashed = pwd_context.hash(u.password)
    cur.execute("INSERT INTO users (name, email, password, role) VALUES (?, ?, ?, ?)",
                (u.name, u.email, hashed, u.role))
    conn.commit()
    conn.close()
    return {"message": "User added"}

@app.post("/add-study-log")
def add_study_log(log: StudyLogIn, current_user: int = Depends(get_current_user)):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT role FROM users WHERE id = ?", (current_user,))
    user = cur.fetchone()
    if not user or user[0] != "student":
        conn.close()
        raise HTTPException(403, "Only students can add logs")

    cur.execute("""
        INSERT INTO study_log
        (user_id, study_hours, social_media_hours, sleep_hours, stress_level, exam_score, notes)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (current_user, log.study_hours, log.social_media_hours, log.sleep_hours,
          log.stress_level, log.exam_score, log.notes))

    conn.commit()
    conn.close()
    return {"message": "Study log added successfully"}

@app.get("/study-logs/{user_id}")
def get_study_logs(user_id: int):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id FROM users WHERE id = ? AND role = 'student'", (user_id,))
    if not cur.fetchone():
        conn.close()
        raise HTTPException(400, "Invalid student")

    cur.execute("""
        SELECT id, date, study_hours, social_media_hours, sleep_hours, stress_level, exam_score, notes
        FROM study_log WHERE user_id = ? ORDER BY date DESC
    """, (user_id,))
    rows = cur.fetchall()
    conn.close()
    return [{
        "id": r[0], "date": str(r[1]), "study_hours": r[2], "social_media_hours": r[3],
        "sleep_hours": r[4], "stress_level": r[5], "exam_score": r[6], "notes": r[7]
    } for r in rows]

Writing backend.py


In [6]:
import nest_asyncio, uvicorn, threading
from pyngrok import ngrok

nest_asyncio.apply()
ngrok.kill()

public_url = ngrok.connect(8000)
print(" FastAPI Public URL:", public_url.public_url)

def run_backend():
    uvicorn.run("backend:app", host="0.0.0.0", port=8000)

threading.Thread(target=run_backend).start()

 FastAPI Public URL: https://unabetted-lorrine-intendedly.ngrok-free.dev


In [7]:
%%writefile frontend.py
import streamlit as st
import requests
import pandas as pd

# === Auto backend URL ===
try:
    BACKEND_URL = st.session_state.fastapi_url
except:
    BACKEND_URL = "http://localhost:8000"

st.set_page_config(page_title="Study Track", layout="wide")

st.markdown("""
<style>
    /* Main font and colors */
    .main > div {padding-top: 2rem;}
    .css-1d391kg {font-family: 'Segoe UI', 'Helvetica Neue', sans-serif;}

    /* Header styling */
    h1, h2, h3, h4 {color: #1e4d7a; font-weight: 600;}

    /* Primary accent color: Calm Teal */
    .stButton > button {
        background-color: #2c7fb8 !important;
        color: white !important;
        border-radius: 8px !important;
        border: none !important;
        font-weight: 500 !important;
        padding: 0.6rem 1.2rem !important;
        height: 48px !important;
    }
    .stButton > button:hover {
        background-color: #236a9a !important;
    }

    /* Secondary buttons */
    div[data-testid="stForm"] .stButton > button {
        background-color: #f0f4f8 !important;
        color: #1e4d7a !important;
    }
    div[data-testid="stForm"] .stButton > button:hover {
        background-color: #d9e2ec !important;
    }

    /* Success & Info messages */
    .stSuccess {background-color: #e8f5e9; border-left: 5px solid #4caf50;}
    .stInfo {background-color: #e3f2fd; border-left: 5px solid #2196f3;}
    .stError {background-color: #ffebee; border-left: 5px solid #f44336;}

    /* Card-like sections */
    .css-1y0t9e3 {  /* st.container */
        background-color: var(--background-color);
        padding: 1.5rem;
        border-radius: 12px;
        border: 1px solid #e0e5eb;
        box-shadow: 0 2px 8px rgba(0,0,0,0.05);
    }

    /* Sidebar */
    .css-1l02z1b {background-color: #f8fafc;}
    section[data-testid="stSidebar"] {border-right: 1px solid #e2e8f0;}

    /* Dataframe styling */
    .stDataFrame {border-radius: 10px; overflow: hidden;}
    .stDataFrame > div {border: 1px solid #e0e5eb;}

    /* Metrics */
    .stMetric > div {background-color: #f0f7ff; border-radius: 10px; padding: 1rem;}
</style>
""", unsafe_allow_html=True)

st.title("Study Track")
st.markdown("### Track your study habits. Improve your focus. Achieve your goals.")

if "user" not in st.session_state:
    st.session_state.user = None

def call(endpoint, data=None):
    try:
        headers = {}
        if st.session_state.user:
            headers["x-user-id"] = str(st.session_state.user["id"])

        url = f"{BACKEND_URL}{endpoint}"
        if data is None:
            r = requests.get(url, headers=headers, timeout=10)
        else:
            r = requests.post(url, json=data, headers=headers, timeout=10)

        if r.status_code == 200:
            return r.json()
        else:
            detail = r.json().get("detail") if r.content and "application/json" in r.headers.get("content-type", "") else r.text
            return {"error": r.text, "detail": detail}
    except Exception as e:
        return {"error": "Connection failed", "exception": str(e)}

# === Sidebar Menu ===
menu = st.sidebar.radio("Navigation", ["Login", "Signup"], label_visibility="collapsed")

# === LOGIN ===
if menu == "Login":
    st.markdown("#### Welcome Back")
    st.markdown("*Log in to continue tracking your progress.*")

    with st.container():
        with st.form("login_form", clear_on_submit=False):
            email = st.text_input("Email Address", placeholder="you@example.com")
            pwd = st.text_input("Password", type="password")
            role = st.selectbox("Login as", ["student", "admin"])

            col1, col2 = st.columns([1, 1])
            with col2:
                submitted = st.form_submit_button(" Login", use_container_width=True, type="primary")

            if submitted:
                res = call("/login", {"email": email, "password": pwd, "role": role})
                if res and res.get("message") == "Login OK":
                    st.session_state.user = res["user"]
                    st.success(f"Welcome back, **{res['user']['name']}**!")
                    st.rerun()
                else:
                    st.error(res.get("detail") or "Invalid email, password, or role.")

# === SIGNUP ===
elif menu == "Signup":
    st.markdown("#### Create Your Account")
    st.markdown("*Start your journey to better study habits today.*")

    with st.container():
        with st.form("signup_form"):
            name = st.text_input("Full Name", placeholder="John Doe")
            email = st.text_input("Email Address", placeholder="john@example.com")
            pwd = st.text_input("Choose Password", type="password", help="Use a strong password")

            col1, col2 = st.columns([1, 1])
            with col2:
                submitted = st.form_submit_button("Create Account", use_container_width=True, type="primary")

            if submitted:
                res = call("/signup", {"name": name, "email": email, "password": pwd, "role": "student"})
                if "successful" in res.get("message", "").lower():
                    st.success("Account created successfully!  Please log in.")
                else:
                    st.error(res.get("detail") or "Registration failed. Try another email.")

# === MAIN DASHBOARD (Authenticated) ===
if st.session_state.user:
    role = st.session_state.user["role"].title()
    name = st.session_state.user["name"]

    st.sidebar.success(f"**{role}**: {name}")
    if st.sidebar.button("Logout", use_container_width=True):
        st.session_state.user = None
        st.success("Logged out successfully.")
        st.rerun()

    # === ADMIN DASHBOARD ===
    if st.session_state.user["role"] == "admin":
        tab1, tab2 = st.tabs([" Students Overview", " Add New User"])

        with tab1:
            st.markdown("### All Registered Students")
            data = call("/students")
            if not data or "error" in data:
                st.info("No students found or unable to load data.")
            else:
                df = pd.DataFrame(data)
                st.dataframe(df[["name", "email"]], use_container_width=True, hide_index=True)

                choice = st.selectbox("View detailed study logs", ["-- Select a student --"] + df["name"].tolist())
                if choice != "-- Select a student --":
                    sid = df[df["name"] == choice]["id"].iloc[0]
                    logs = call(f"/study-logs/{sid}")
                    if logs and len(logs) > 0:
                        log_df = pd.DataFrame(logs)
                        log_df["date"] = pd.to_datetime(log_df["date"]).dt.strftime("%b %d, %Y • %I:%M %p")
                        st.markdown(f"####  {choice}'s Study History")
                        st.dataframe(log_df[["date", "study_hours", "social_media_hours", "sleep_hours", "stress_level", "exam_score", "notes"]],
                                   use_container_width=True, hide_index=True)
                    else:
                        st.info("No study logs recorded yet.")

        with tab2:
            st.markdown("### Add New User (Admin Only)")
            with st.form("add_user_form"):
                n = st.text_input("Full Name")
                e = st.text_input("Email Address")
                p = st.text_input("Password", type="password")
                r = st.selectbox("User Role", ["student", "admin"])

                if st.form_submit_button("Add User", use_container_width=True, type="primary"):
                    resp = call("/add-user", {"name": n, "email": e, "password": p, "role": r})
                    if resp and "added" in resp.get("message", "").lower():
                        st.success("User added successfully!")
                        st.rerun()
                    else:
                        st.error(resp.get("detail") or "Failed to add user.")

    # === STUDENT DASHBOARD ===
    else:
        tab1, tab2 = st.tabs([" Log Today's Progress", " My Study History"])

        with tab1:
            st.markdown("#### Log Your Daily Study Session")
            st.markdown("*Be honest — consistency beats perfection.*")

            with st.form("study_form", clear_on_submit=True):
                col1, col2 = st.columns(2)
                with col1:
                    study_h = st.number_input("Study Hours", min_value=0.0, max_value=24.0, value=4.0, step=0.5)
                    social_h = st.number_input("Social Media Hours", min_value=0.0, max_value=24.0, value=1.0, step=0.5)
                    sleep_h = st.number_input("Sleep Hours", min_value=0.0, max_value=24.0, value=8.0, step=0.5)
                with col2:
                    stress = st.slider("Stress Level", 1, 10, 5, help="1 = Relaxed, 10 = Overwhelmed")
                    exam_s = st.number_input("Exam/Quiz Score (if any)", min_value=0.0, max_value=100.0, step=0.5, format="%.1f", value=None, placeholder="Leave blank if none")
                    notes = st.text_area("Notes & Reflections", placeholder="What went well? What to improve?", height=100)

                submitted = st.form_submit_button(" Save Today's Log", use_container_width=True, type="primary")
                if submitted:
                    payload = {
                        "study_hours": float(study_h),
                        "social_media_hours": float(social_h),
                        "sleep_hours": float(sleep_h),
                        "stress_level": int(stress),
                        "exam_score": float(exam_s) if exam_s and exam_s > 0 else None,
                        "notes": notes.strip() if notes.strip() else None
                    }
                    resp = call("/add-study-log", payload)
                    if resp and resp.get("message") == "Study log added successfully":
                        st.success("Log saved successfully! Keep it up! ")
                    else:
                        st.error("Failed to save log. Please try again.")

        with tab2:
            st.markdown("### Your Study Journey")
            user_id = st.session_state.user["id"]
            logs = call(f"/study-logs/{user_id}")

            if logs and len(logs) > 0:
                df = pd.DataFrame(logs)
                df["date"] = pd.to_datetime(df["date"]).dt.strftime("%b %d, %Y • %I:%M %p")
                df = df.sort_values("date", ascending=False)

                st.dataframe(df[["date", "study_hours", "social_media_hours", "sleep_hours", "stress_level", "exam_score", "notes"]],
                            use_container_width=True, hide_index=True)
            else:
                st.info("No logs yet. Your journey starts today — go log your first session!")

# === Debug Sidebar (collapsible) ===
with st.sidebar.expander(" Backend Debug", expanded=False):
    st.caption(f"Backend URL: `{BACKEND_URL}`")
    st.caption(f"User ID: {st.session_state.user['id'] if st.session_state.user else 'Not logged in'}")
    if "ngrok" in BACKEND_URL.lower():
        st.success("Connected via ngrok")

Writing frontend.py


In [8]:
!pkill -f streamlit
!streamlit run frontend.py --server.port 8501 --server.headless true &> logs.txt &
from pyngrok import ngrok
import time
time.sleep(8)
url = ngrok.connect(8501, bind_tls=True)
print("STREAMLIT PUBLIC URL:")
print(url.public_url)

STREAMLIT PUBLIC URL:
https://unabetted-lorrine-intendedly.ngrok-free.dev


In [ ]:
!pkill -f ngrok
!pkill -f uvicorn